In [1]:
import pandas as pd

site_web = "datasets/site_web.parquet"

df = pd.read_parquet(site_web)

In [1]:
import pandas as pd
import plotly.graph_objects as go
link = "datasets/actors_movies.parquet"
actors = pd.read_parquet(link)

In [4]:
link = "datasets/movies_cleaned.parquet"
movies = pd.read_parquet(link)

In [8]:
total_genres = movies.explode("titre_genres").groupby("titre_genres")["rating_avg"].mean().reset_index()

In [9]:
total_genres

,titre_genres,rating_avg
0,Action,6.996991
1,Adventure,7.048584
2,Animation,7.197783
3,Comedy,6.966282
4,Crime,7.042778
5,Drama,7.125658
6,Family,7.052342
7,Fantasy,7.055268
8,History,7.201257
9,Horror,6.790253


In [ ]:
# Note moyenne par décennie
average_note = (
    actors.groupby("cuts", observed=True)["rating_avg"]
    .mean()
    .reset_index(name="Notes")
)
fig1 = go.Figure()
fig1.add_trace(
    go.Bar(
        x=average_note["cuts"],
        y=average_note["Notes"],
        showlegend=False,
        marker=dict(
            color="royalblue", line=dict(color="black", width=1)
        ),
        name="Notes moyennes"
    )
)
fig1.update_layout(
    title="Notes moyennes par décennie",
    xaxis_title="Décennie",
    yaxis_title="Notes moyennes",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="left",
        x=0.01,
    ),
)
fig1.show()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
import pandas as pd

def get_titre_from_index(df: pd.DataFrame, idx: int) -> str:
    """
    Récupère le titre correspondant à un index donné dans un DataFrame.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame contenant les titres.
    idx : int
        Index du titre à récupérer.

    Returns
    -------
    str
        Titre correspondant à l'index fourni.
    """
    return df[df.index == idx]["titre_str"].values[0]

def knn_algo(df: pd.DataFrame, titre: str, top: int = 5) -> list:
    index = df[df["titre_str"] == titre].index[0]
    cv = CountVectorizer()
    count_matrix = cv.fit_transform(
        df["one_for_all"]
    )
    knn_model = NearestNeighbors(
        metric="cosine",
        algorithm="brute",
        n_jobs=-1,
    ).fit(count_matrix)
    _, indices = knn_model.kneighbors(
        count_matrix[index], n_neighbors=top + 1
    )
    result = []
    for idx in indices.flatten()[1:]:
        recommandations = get_titre_from_index(df, idx)
        result.append(recommandations)
    return result

In [ ]:
def clean_dup(df: pd.DataFrame) -> pd.DataFrame:
    """
    Nettoie les doublons dans une colonne spécifique d'un DataFrame en ajoutant
    la date entre parenthèses.

    Parameters
    ----------
    df : pd.DataFrame
        Le DataFrame à nettoyer.

    Returns
    -------
    pd.DataFrame
        DataFrame avec les doublons nettoyés.
    """
    condi = df["titre_str"].duplicated(keep=False)
    df.loc[condi, "titre_str"] = (
        df.loc[condi, "titre_str"]
        + " "
        + "("
        + df.loc[condi, "date"].astype(str)
        + ")"
    )
    return df

In [ ]:
machine_learning = "datasets/machine_learning_final.parquet"
site_web = "datasets/site_web.parquet"
df_ml = pd.read_parquet(machine_learning)
df_ml = clean_dup(df_ml)
df_sw = pd.read_parquet(site_web)
df_sw = clean_dup(df_sw)

selectvalue = input("Nom du film :")

selected_movie = df_sw[df_sw["titre_str"] == selectvalue]

recommended = knn_algo(df_ml, selectvalue, 7)

In [1]:
import pandas as pd

pd.__version__

'1.5.3'